### Predicciones con videos que no pertenecen al grupo de videos de entrenamiento

In [1]:
import cv2
import os
import re
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def video_to_array(filename):
    cap = cv2.VideoCapture(filename)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_array = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('float32'))

    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, video_array[fc] = cap.read()
        fc += 1

    cap.release()

    return video_array

In [2]:
# Load the model
filepath = './saved_model'

model = load_model(filepath, compile = True)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 370, 16, 30, 32)   2624      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 185, 8, 15, 32)    0         
_________________________________________________________________
dropout (Dropout)            (None, 185, 8, 15, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 183, 6, 13, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 91, 3, 6, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 91, 3, 6, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 104832)            0

In [12]:
# Test con varios videos aleatorios de X e Y
directorypath = 'C:/Users/tania/0_Plabs3D/clasificacion_videos/Dataset_test/Predictions'

samples_to_predict = []
labels_verificacion = []

for root, dirnames, filenames in os.walk(directorypath):
    for filename in filenames:
        if 'X' in filename:
            labels_verificacion.append(0)
        elif 'Y' in filename:
            labels_verificacion.append(1)

        video = video_to_array(directorypath + '/' + filename)
        video = video/255  # Pasar a negro o blanco (dentro del intervalo [0,1])
        
        samples_to_predict.append(video)

samples_to_predict = np.array(samples_to_predict)

print(samples_to_predict.shape)

(128, 372, 18, 32, 3)
La prediccion deberia de darnos:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
predictions = model.predict(samples_to_predict)
classes = np.argmax(predictions, axis = 1)
print('Etiquetas correctas  : ', labels_verificacion)
print('Prediccion del modelo: ', classes)

score = 0
for classe, label in zip(classes, labels_verificacion):
    if classe == label:
        score += 1

total = len(classes)

print('El modelo ha acertado %s de %s de las predicciones' % (score, total))

Etiquetas correctas  :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Prediccion del modelo:  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
El modelo ha acertado 126 de 128 de las predicciones


In [5]:
# Test con videos reales
directorypath = 'C:/Users/tania/0_Plabs3D/clasificacion_videos/videos_reales/Modified_Videos'
samples_to_predict = []
labels_verificacion = []

for root, dirnames, filenames in os.walk(directorypath):
    for filename in filenames:
        if 'X' in filename:
            labels_verificacion.append(0)
        elif 'Y' in filename:
            labels_verificacion.append(1)
        else:
            labels_verificacion.append(2)

        video = video_to_array(directorypath + '/' + filename)
        video = video/255  # Pasar a negro o blanco (dentro del intervalo [0,1])
        
        samples_to_predict.append(video)

samples_to_predict = np.array(samples_to_predict)

print(samples_to_predict.shape)

# print('La prediccion deberia de darnos: ', labels_verificacion)

(14, 372, 18, 32, 3)


In [6]:
predictions = model.predict(samples_to_predict)
classes = np.argmax(predictions, axis = 1)
print('Etiquetas correctas  : ', labels_verificacion)
print('Prediccion del modelo: ', classes)

score = 0
for classe, label in zip(classes, labels_verificacion):
    if classe == label:
        score += 1

total = len(classes)

print('El modelo ha acertado %s de %s de las predicciones' % (score, total))

Etiquetas correctas  :  [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
Prediccion del modelo:  [0 0 0 0 0 0 0 1 0 1 0 0 1 1]
El modelo ha acertado 7 de 14 de las predicciones
